In [1]:
# !pip install -r requirements.txt

In [2]:
# import os
# os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import pandas as pd
import numpy as np
import geopandas as gpd
# import fiona
import datetime as dt

from utils import *

import calitp
from calitp.tables import tbl
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### (Old) Metric

See accessible_transit_metrics notebook for new version of this metric; leaving this in case census disability data is useful in the future.

The % of [wheelchair-using] Californians that are within 1/4 mi of a transit stop that is indicated to be explicitly accessible in a static GTFS feed and served by at least one explicitly accessible trip.

In [3]:
accessible_stops_trips = get_accessible_stops_and_trips()

### Census Data

In [4]:
variable_range = ['0' + str(x) if x < 10 else str(x) for x in range(1,34)]

In [5]:
## https://api.census.gov/data/2019/acs/acs5/variables.html
ambulatory_disability_vars = [f'B18105_0{x}E' for x in variable_range]

In [6]:
ambulatory_disability_vars = ','.join(ambulatory_disability_vars)

In [7]:
ambulatory_disability_df = get_census_ca_counties(ambulatory_disability_vars)

In [8]:
ambulatory_disability_df.head(3)

,B18105_001E,B18105_002E,B18105_003E,B18105_004E,B18105_005E,B18105_006E,B18105_007E,B18105_008E,B18105_009E,B18105_010E,...,B18105_027E,B18105_028E,B18105_029E,B18105_030E,B18105_031E,B18105_032E,B18105_033E,state,county,tract
0,1901,1063,208,0,208,310,8,302,366,26,...,354,75,29,46,48,15,33,6,47,503
1,6111,3287,871,30,841,996,0,996,1079,94,...,875,202,41,161,144,68,76,6,47,504
2,6099,3242,887,36,851,833,0,833,1090,83,...,979,198,29,169,252,72,180,6,47,505


In [9]:
ambulatory_disability_df['total_pop'] = ambulatory_disability_df['B18105_001E']
disability_cols = ['B18105_004E', 'B18105_007E', 'B18105_010E', 
                  'B18105_013E', 'B18105_016E', 'B18105_020E',
                   'B18105_023E', 'B18105_026E', 'B18105_029E',
                   'B18105_032E'
                  ]
ambulatory_disability_df['total_disabled'] = ambulatory_disability_df[disability_cols].sum(axis=1)
ambulatory_disability_summary = ambulatory_disability_df[['state', 'county', 'tract', 'total_pop', 'total_disabled']]

In [10]:
ambulatory_disability_summary.head(3)

,state,county,tract,total_pop,total_disabled
0,6,47,503,1901,180
1,6,47,504,6111,423
2,6,47,505,6099,387


## Block Group Pop

In [11]:
total_pop_var = 'B01001_001E'

In [12]:
blockgrp_pop = get_census_ca_counties(total_pop_var, 'block%20group')

In [13]:
blockgrp_pop = blockgrp_pop.rename(columns={'B01001_001E': 'block_grp_pop'})

In [14]:
blockgrp_pop.head(3)

,block_grp_pop,state,county,tract,block group
0,1388,6,47,1200,2
1,1531,6,47,1200,4
2,1405,6,47,1200,1


In [15]:
census_joined = blockgrp_pop >> inner_join(_, ambulatory_disability_summary, on=['county', 'tract'])

In [16]:
census_joined.head(3)

,block_grp_pop,state_x,county,tract,block group,state_y,total_pop,total_disabled
0,1388,6,47,1200,2,6,5012,428
1,1531,6,47,1200,4,6,5012,428
2,1405,6,47,1200,1,6,5012,428


In [17]:
census_joined['pct_disabled'] = census_joined['total_disabled'] / census_joined['total_pop']
census_joined['block_grp_disabled_est'] = (census_joined['block_grp_pop'] * census_joined['pct_disabled']).round(0)
census_joined = census_joined.dropna()
census_joined['block_grp_disabled_est'] = census_joined['block_grp_disabled_est'].astype('int64')

In [18]:
census_joined.head(3)

,block_grp_pop,state_x,county,tract,block group,state_y,total_pop,total_disabled,pct_disabled,block_grp_disabled_est
0,1388,6,47,1200,2,6,5012,428,0.085395,119
1,1531,6,47,1200,4,6,5012,428,0.085395,131
2,1405,6,47,1200,1,6,5012,428,0.085395,120


In [19]:
## .25mi buffer on each accessible stop served by an accessible trip
accessible_stops_trips.geometry = accessible_stops_trips.buffer(400)